In [ ]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout, Reshape, Permute, Activation, BatchNormalization, \
    Input
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, Callback
import numpy as np
from keras.wrappers.scikit_learn import KerasRegressor
import matplotlib.pyplot as plt
import pickle as pkl
%matplotlib inline

In [ ]:
# load data
# Xtrain = np.load('./data/dataset256/Xtrain.npy')
# Xtest = np.load('./data/dataset256/Xtest.npy')
Ytrain = np.load('./data/dataset256/Ytrain_cls.npy')
Ytest = np.load('./data/dataset256/Ytest_cls.npy')

In [ ]:
# netowrk settings
batch_size = 64
nb_epoch = 5

In [ ]:
def MyNet():
    model = Sequential()
    model.add(Convolution2D(16, 3, 3, border_mode='same', input_shape=(256, 256,3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(4, 4)))
    
    model.add(Convolution2D(32, 3, 3))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(4, 4)))
    
    model.add(Convolution2D(64, 3, 3, border_mode='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(4, 4)))

    model.add(Flatten())
    
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    
    model.add(Dense(1))
    
    model.compile(loss='mean_squared_error', optimizer='adam')
    
    return model


In [ ]:
model = MyNet()
model.summary()

In [ ]:
#train just 1000 samples, 3 epochs
# if False:
checkpoint = ModelCheckpoint(filepath='./cnn-training-checkpoint/checkpoint-{epoch:02d}-{val_loss:.2f}.hdf5')
history = model.fit(x=Xtrain[:100], y=Ytrain[:100], 
          nb_epoch=nb_epoch, batch_size=batch_size,validation_split=0.1, verbose=1, callbacks=[checkpoint])
with open('./cnn-training-checkpoint//history.pkl', 'wb') as f:
    pkl.dump(history.history, f)

In [ ]:
plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()

In [ ]:
model.evaluate(x=Xtest, y=Ytest,batch_size=batch_size)